In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import cProfile
from collections import deque, Counter
from time import time

path = '/home/mariano/Descargas/VID_20170720_132629833.mp4'

from calculo.video import centroid  # Cython

def center_of_square(contour):
    return centroid(*cv2.boundingRect(contour))

def image_proccess(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    binary = cv2.threshold(gray, 240., 255., cv2.THRESH_BINARY)[1]
    contours= cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[1]
    mark = np.array(map(center_of_square, contours))
    return mark.shape[0], mark


In [3]:

def roi_center(array, amp=1.15):
    y1 = array[0][1]
    y2 = array[-1][1]
    dy = (y1 - y2)*amp
    center = (y1 + y2) / 2
    return(center - dy, center +  dy)  # límite superior, límite inferior

def partitionate(markers, n_expected, kroi=False):
    if kroi:
        Y = markers[:, 1]
        G0 = markers[Y < kroi[0]]
        G1 = markers[np.logical_and(Y > kroi[0], Y < kroi[1])]
        G2 = markers[Y > kroi[1]]
    else:
        G0 = markers[5:, :]
        G1 = markers[3:5, :]
        G2 = markers[:3, :]
    n_obtained = (G0.shape[0], G1.shape[0], G2.shape[0])
    boolean_interpolate = [a != b for a, b in zip(n_expected, n_obtained)]
    to_interpolate = [v for v, b in zip((0, 1, 2), boolean_interpolate) if b]
    return(to_interpolate, (G0, G1, G2))

In [5]:
%%time
# En este bloque se abre el video y se procesan las imágenes en busqueda de los
# marcadores.
# Solo se agregan al arreglo de datos(list) los arreglos de marcadores que se 
# delimitan en un intervalo cerrado cuyos extremos(arreglo de marcadores tambien)
# poseen 7 marcadores.
vid = cv2.VideoCapture(path)
is_frame, frame = vid.read()
expected_markers = (2, 2, 3)
M = sum(expected_markers)
safe_None = 25
last_M = 0
active = False
count_non_data = 0
data = []
interp = []
i = 0

while is_frame:
    m, mark = image_proccess(frame)
    if mark.any():
        if m == M:
            active = True
            last_M = i
            count_non_data = 0
        if active:
            if m != M:
                to_interp, groups = partitionate(mark, expected_markers, kr)
                data.append(groups)
                interp.append((i, to_interp))
            else:
                __, groups = partitionate(mark, expected_markers)
                kr = roi_center(groups[1])
                data.append(groups)
        else:
            data.append(None)
    else:
        count_non_data += 1
        if active and count_non_data < safe_None:
            data.append(mark)
            interp.append(i)
        else:
            if active:
                backwards = i - last_M
                for __ in range(backwards - 1):
                    data.pop()
                    interp.pop()
                data += [None, ]*backwards
                active = False
            else:
                data.append(None)

    is_frame, frame = vid.read()
    i += 1

CPU times: user 44.9 s, sys: 424 ms, total: 45.3 s
Wall time: 17.8 s


In [6]:
interp

[(13, [0]),
 (14, [0]),
 (15, [0]),
 (16, [0]),
 (22, [2]),
 (34, [0]),
 (35, [0]),
 (40, [2]),
 (52, [0]),
 (54, [0]),
 (55, [0]),
 (56, [2]),
 (57, [2]),
 (371, [0]),
 (372, [0]),
 (375, [2]),
 (376, [2]),
 (389, [0]),
 (390, [0]),
 (405, [0]),
 (408, [2]),
 (409, [2]),
 (410, [2]),
 (411, [2]),
 (746, [0]),
 (747, [0]),
 (750, [2]),
 (761, [0]),
 (762, [0]),
 (763, [0]),
 (768, [2]),
 (779, [0]),
 (780, [0]),
 (782, [2]),
 (784, [2]),
 (785, [2]),
 (798, [0]),
 (799, [0]),
 (800, [0]),
 (801, [2]),
 (1173, [2]),
 (1174, [2]),
 (1185, [0]),
 (1186, [0]),
 (1188, [2]),
 (1204, [0]),
 (1205, [0]),
 (1218, [0]),
 (1219, [0]),
 (1225, [2]),
 (1226, [2])]

In [130]:
F = 2, 2, 3
G = 1, 2, 4
Bl = [a != b for a, b in zip(F, G)]


[0, 2]

In [17]:
#Necesito corregir los marcadores de tobillo.
pie_y = map(lambda x: x[0, 1] if isinstance(x, np.ndarray) else None, markers)

def diff(array):
    prev = array[0]
    vel = []
    for new in array:
        if new is None:
            vel.append(None)
        else:
            vel.append(new - prev)
            prev = new
    return vel

p1 = map(lambda x: x[0, 1] if isinstance(x, np.ndarray) else None, ankle)

vel = diff(p1)

IndexError: too many indices for array

In [5]:
def sort_foot_markers(ankle_array, direction=False):
    if ankle_array is None:
        return ankle_array
    else:
        x1, x2, refer = ankle_array[:, 0]
        if direction:
            if direction > 0:
                if x1 < x2:
                    talon = ankle_array[0]
                    antepie = ankle_array[1]
                else:
                    talon = ankle_array[1]
                    antepie = ankle_array[0]
            elif direction < 0:
                if x1 < x2:
                    talon = ankle_array[1]
                    antepie = ankle_array[0]
                else:
                    talon = ankle_array[0]
                    antepie = ankle_array[1]
        else:
            values = ((abs(refer - x1), 0),
                      (abs(refer - x2), 1))
            talon = ankle_array[min(values)[1]]
            antepie = ankle_array[max(values)[1]]            
    maleolo = ankle_array[-1]
    return np.array((maleolo, talon, antepie))


def get_direction(ankle_array):
    x_talon = ankle_array[1, 0]
    x_antepie = ankle_array[2, 0]
    if x_talon < x_antepie:
        return 1
    else:
        return -1

In [6]:
def set_direction(ankle_velocities, none_limit=25):
    values = []
    buff = []
    for i, vel in enumerate(ankle_velocities):
        if vel == 0:
            # establecer la dirección. Agregar a la lista de salida.
            ordered = sort_foot_markers(ankle[i])
            direction = get_direction(ordered)
            values.append((i, direction))
        elif vel is None:
            # agregar a la lista de salida
            values.append((i, None))
        else:
            #guardar en buffer
            buff.append((i, 0))
    directions = []
    for i, val in values + buff:
        directions.insert(i, val)
    # En la lista directions existen valores de direcición, de velocidad != 0
    # y None. Para una lista homogénea de direcciones se vuelve a procesar.
    none_values = 0
    rare_indices = []
    dire_indices = []
    dire_values = []
    for i, val in enumerate(directions):
        if val is None:
            if none_values < none_limit:
                # Se puede estar dentro de la imagen que tiene marcadores
                rare_indices.append(i)
                none_values += 1
            else:
                # La imagen ya no tiene marcadores (según el límite de
                # valores None que establece el usuario*; Si hay valores de
                # dirección se reemplazan en el arreglo original.
                if dire_values != []:
                    # Como existe mas de un valor que del otro en direcciones
                    # (ver mas abajo), el signo de la suma establece la
                    # dirección correcta (y final) de los intervalos del
                    # arreglo.
                    freq = 1 if sum(dire_values) > 0 else -1
                    for j in dire_indices + rare_indices[:-none_limit]:
                        directions.pop(j)
                        directions.insert(j, freq)
                    dire_values = []
                    dire_indices = []
                    rare_indices = []
        elif val == 0:
            # Entonces la dirección es desconocida y estos valores se
            # tienen que reemplazar por el valor de dirección.
            rare_indices.append(i)
            none_values = 0
        else:
            # Si no es None, y tampoco es 0, entonces es un valor de
            # dirección (1 ó -1). Pueden coexistir ambos, porque en
            # la primer lectura de direcciones hay intercambio de
            # marcadores de talon y antepie. Sin embargo como la mayor
            # parte de las direcciones se toman en apoyo, simpre va
            # a haber mas de un valor que del otro.
            dire_values.append(val)
            dire_indices.append(i)
            none_values = 0
    return directions

direction = set_direction(vel)
sorted_ankle = [sort_foot_markers(arr, direction=d) for arr, d in zip(ankle, direction)]

for i in zip(ankle, sorted_ankle):
    print '##############'
    print i[0]
    print i[1]

##############
[[ 227 1045]
 [ 157 1041]
 [ 170  988]]
[[ 170  988]
 [ 157 1041]
 [ 227 1045]]
##############
[[ 227 1045]
 [ 157 1041]
 [ 170  988]]
[[ 170  988]
 [ 157 1041]
 [ 227 1045]]
##############
[[ 227 1045]
 [ 157 1041]
 [ 172  988]]
[[ 172  988]
 [ 157 1041]
 [ 227 1045]]
##############
[[ 228 1045]
 [ 158 1040]
 [ 174  988]]
[[ 174  988]
 [ 158 1040]
 [ 228 1045]]
##############
[[ 228 1045]
 [ 158 1040]
 [ 176  988]]
[[ 176  988]
 [ 158 1040]
 [ 228 1045]]
##############
[[ 228 1045]
 [ 158 1040]
 [ 178  988]]
[[ 178  988]
 [ 158 1040]
 [ 228 1045]]
##############
[[ 229 1045]
 [ 159 1039]
 [ 180  987]]
[[ 180  987]
 [ 159 1039]
 [ 229 1045]]
##############
[[ 230 1045]
 [ 160 1037]
 [ 184  985]]
[[ 184  985]
 [ 160 1037]
 [ 230 1045]]
##############
[[ 231 1045]
 [ 163 1033]
 [ 189  980]]
[[ 189  980]
 [ 163 1033]
 [ 231 1045]]
##############
[[ 234 1044]
 [ 167 1027]
 [ 194  974]]
[[ 194  974]
 [ 167 1027]
 [ 234 1044]]
##############
[[ 239 1044]
 [ 173 1018]
 [ 205  9

None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############


##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None


[[1618  928]
 [1581  977]
 [1641 1022]]
##############
[[1655 1012]
 [1606  957]
 [1649  915]]
[[1649  915]
 [1606  957]
 [1655 1012]]
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
[[1917  989]
 [1885  951]
 [1902  902]]
[[1902  902]
 [1885  951]
 [1917  989]]
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None
None
##############
None


In [7]:
def linear(x, x1, x2, y1, y2):
    num = (x - x1)*(y2 - y1)
    div = (x2 - x1)
    if not div.all():
        return (y2 + y1) / 2
    return num / div + y1

def interpolate(A, B, n_steps):
    n_steps += 1
    x1, y1 = A.T
    x2, y2 = B.T
    dx = (x2 - x1) / n_steps
    interpolated = []
    for n in xrange(1, n_steps):
        X = x1 + dx*n
        Y = linear(X, x1, x2, y1, y2)
        interpolated.append(np.array((X, Y), dtype=int).T)
    return interpolated

def fill_in(array, none_limit=25):
    interpolated = []
    none_indices = []
    none_values = none_limit
    for i, c_arr in enumerate(array):
        if c_arr is None:
            if none_values < none_limit:
                none_indices.append(i)
            else:
                none_indices = []
            none_values += 1
        else:
            if none_indices:
                interp = interpolate(l_arr, c_arr, none_values)
                for par in zip(none_indices, interp):
                    interpolated.append(par)
                none_indices = []
                none_values = 0
            else:
                none_values = 0
            l_arr = c_arr
    for i, arr in interpolated:
        array.pop(i)
        array.insert(i, arr)

In [8]:
from time import sleep
#array = [partition['ankle'] for partition in markers]
array = sorted_ankle

vid = cv2.VideoCapture(path)
n = 0
while True:
    is_frame, frame = vid.read()
    if is_frame is False:
        break
    marker = array[n]
    if marker is not None:
        for arr in marker:
            cv2.circle(frame, (arr[0], arr[1]), 7, (0, 0, 255), -1)
    cv2.imshow(path, cv2.resize(frame, None, fx=.4, fy=.4))
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    n += 1
    sleep(.15)
vid.release()
cv2.destroyAllWindows()